In [1]:
from typing import Any

from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("22"))

puzzle.header()

# Monkey Market

[Open Website](https://adventofcode.com/2024/day/22)

In [2]:
# helper functions
def domain_from_input(input: InputReader) -> list:
    lines = input.lines_as_int()

    return lines


def mix_and_prune(secret: int, result: int) -> int:
    # Mix the value into the secret number
    mixed = secret ^ result
    # Prune the secret number
    return mixed % 16777216


result = mix_and_prune(15, 42)
assert result == 37

result = mix_and_prune(16777216, 100000000)
assert result == 16113920


def forward(secret: int) -> int:
    # Step 1: Multiply by 64
    secret = mix_and_prune(secret, secret * 64)

    # Step 2: Divide by 32 and round down
    secret = mix_and_prune(secret, secret // 32)

    # Step 3: Multiply by 2048
    secret = mix_and_prune(secret, secret * 2048)

    return secret


answers = [
    15887950,
    16495136,
    527345,
    704524,
    1553684,
    12683156,
    11100544,
    12249484,
    7753432,
    5908254
]

start = 123
for answer in answers:
    start = forward(start)
    assert start == answer


In [3]:
# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    starts = domain_from_input(reader)

    if debug:
        display(starts)

    for i in range(2000):
        starts = [forward(s) for s in starts]

    if debug:
        display(starts)

    return sum(starts)


result = part_1(puzzle.get_code_block(1, 10), True)
display(result)
assert result == 37327623

[1, 10, 100, 2024]

[8685429, 4700978, 15273692, 8667524]

37327623

In [4]:
# real case (part 1)
result = part_1(puzzle.input(), False)
print(result)
assert result == 21147129593

21147129593


In [5]:
from functools import cache


# test case (part 2)
@cache
def forward_2(secret: int) -> tuple:
    last_digit = secret % 10
    secret = forward(secret)
    next_digit = secret % 10
    diff = next_digit - last_digit
    return (secret, next_digit, diff)


answers = [-3, 6, -1, -1, 0, 2, -2, 0, -2]

start = 123
for i in range(9):
    start, price, diff = forward_2(start)
    assert diff == answers[i]

start = 1
target = [-2, 1, -1, 3]
history = []
for i in range(2000):
    start, price, diff = forward_2(start)
    history.append(diff)
    # is history equals target
    if history[-4::] == target:
        print(f"Found target at {i}")
        print(f"Price: {price}")
        assert price == 7

    # history is larger than 4, then remove front
    if len(history) > 4:
        history.pop(0)


Found target at 1963
Price: 7


In [6]:
def part_2(reader: InputReader, debug: bool) -> int:
    starts = domain_from_input(reader)

    if debug:
        display(starts)

    def find_price(start: int, target: list) -> int | None:
        # get the history of the start
        hist = history[start]
        # find the target in the history
        for i in range(len(hist) - 4):
            subset = hist[i:i + 4]
            diffs = [x[1] for x in subset]
            if diffs == target:
                return subset[-1][0]
        return None

    history = dict()
    queue = dict()
    # build dictionary for history of each start
    for start in starts:
        next, price, diff = forward_2(start)
        history[start] = [(price, diff)]
        queue[start] = next

    for start in history.keys():
        for _ in range(2000):
            last = queue[start]
            next, price, diff = forward_2(last)
            history[start].append((price, diff))
            queue[start] = next

    # build diff histories
    diff_histories = dict()
    for start in history.keys():
        diff_histories[start] = [x[1] for x in history[start]]

    def unique_series_of_4(history: list) -> set:
        series = set()
        for i in range(len(history) - 4):
            next = history[i:i + 4]
            next = [str(x) for x in next]
            series.add(",".join(next))
        return series

    def find_common_series(history: dict) -> dict:
        patterns = {}
        for start in history.keys():
            for next in unique_series_of_4(history[start]):
                if next in patterns:
                    patterns[next] += 1
                else:
                    patterns[next] = 1
        return patterns

    common = find_common_series(diff_histories)
    candidates = {}
    for pattern, count in common.items():
        if count in candidates:
            candidates[count].append(pattern)
        else:
            candidates[count] = [pattern]

    keys = list(candidates.keys())
    keys.sort(reverse=True)
    keys = keys[:20]

    max = 0
    for key in keys:
        print(f"Checking {key}")
        sets = candidates[key]
        for target in sets:
            target = [int(x) for x in target.split(",")]
            total = 0
            for start in history.keys():
                price = find_price(start, target)
                if price:
                    total += price
            if total > max:
                max = total
                if debug:
                    print(f"New max: {max}")

    return max


result = part_2(InputReader("1\n2\n3\n2024"), True)
display(result)
assert result == 23

[1, 2, 3, 2024]

Checking 3
New max: 15
New max: 20
New max: 21
New max: 22
New max: 23
Checking 2


Checking 1


23

In [7]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)
assert result == 2445

Checking 454


Checking 443


Checking 436


Checking 430


Checking 429


...

Checking 407


Checking 406


Checking 405


Checking 404


2445

In [8]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="Some might say it would be... bananas.">ridiculous</span> (Some might say it would be... bananas.)